In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI


In [3]:
loader = CSVLoader(file_path="../data/data_csv_json.csv")
data = loader.load()


In [4]:
print(data)

[Document(metadata={'source': '../data/data_csv_json.csv', 'row': 0}, page_content='Supervisor\'s name: Jarosław Drapała\ninterests: mathematical modelingdynamical systemsmachine learningexploratory data analysis\nresearch papers: {"Impairments of working memory in schizophrenia and bipolar disorder: the effect of history of psychotic symptoms and different aspects of cognitive task demands": "Comparisons of cognitive impairments between schizophrenia (SZ) and bipolar disorder (BPD) have produced mixed results. We applied different working memory (WM) measures (Digit Span Forward and Backward, Short-delay and Long-delay CPT-AX, N-back) to patients with SZ (n = 23), psychotic BPD (n = 19) and non-psychotic BPD (n = 24), as well as to healthy controls (HC) (n = 18) in order to compare the level of WM impairments across the groups. With respect to the less demanding WM measures (Digit Span Forward and Backward, Short-delay CPT-AX), there were no between group differences in cognitive perf

In [5]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(data)
db = FAISS.from_documents(documents, OpenAIEmbeddings())

In [6]:
retriever = db.as_retriever(search_kwargs={"k": 5})

In [7]:
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(temperature=0.2),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

In [8]:
prompt_template = """
You are an expert in providing information about thesis supervisors at Politechnika Wrocławska.
When given a user question about the most suitable thesis supervisor for their project based on their reasearch papers and intrests, provide the supervisor's name along with titles of their research papers.

User Question:
{question}

Response:
"""

In [9]:
question = "Who should I pick as promotor for my engineering thesis 'Train Rescheduling and Track Closure Optimization'"
formatted_prompt = prompt_template.format(question=question)
print(qa_chain.invoke(formatted_prompt)["result"])


Based on your research topic, I would recommend considering Dariusz Gąsior as your thesis supervisor. He has a strong interest in transportation and optimization, as seen in his research papers "Pareto-optimal Nash equilibrium in capacity allocation game for self-managed networks" and "An Algorithm for Rescheduling of Trains under Planned Track Closures". These papers demonstrate his expertise in solving complex optimization problems related to transportation systems. Additionally, his experience in self-managed networks may also be applicable to your research topic.


In [10]:
question = "Who should I pick as promotor for my engineering thesis 'Heuristic Methods for Carpooling Optimization: Balancing Economic, Comfort, Safety, and Social Criteria'"
formatted_prompt = prompt_template.format(question=question)
print(qa_chain.invoke(formatted_prompt)["result"])


Based on your research topic and interests, the most suitable thesis supervisor for you would be Grzegorz Filcek. Some of his research papers that align with your topic and interests include "A heuristic algorithm for solving a Multiple Criteria Carpooling Optimization (MCCO) problem" and "ADAPTIVE DECISION SUPPORT SYSTEM FOR AUTOMATIC PHYSICAL EFFORT PLAN GENERATION\u2014DATA-DRIVEN APPROACH".


In [11]:
question = "Who should I pick as promotor for my engineering thesis 'The Role of Recomendation Systems in Ecommerce?'"
formatted_prompt = prompt_template.format(question=question)
print(qa_chain.invoke(formatted_prompt)["result"])


Based on your research interests and the topic of your thesis, I would recommend considering Grzegorz Filcek as your supervisor. He has a strong background in multiple criteria optimization and mathematical modeling, which could be applicable to your research on recommendation systems in ecommerce. Some of his relevant research papers include "A heuristic algorithm for solving a Multiple Criteria Carpooling Optimization (MCCO) problem" and "Pareto-optimal Nash equilibrium in capacity allocation game for self-managed networks."
